In [73]:
import requests
from bs4 import BeautifulSoup
import os
import sys
import time
import json
import random
import re
import json

In [74]:
payload = {'year':'all', 'model':'warning'}
response = requests.post('http://rdc28.cwb.gov.tw/TDB/ctrl_typhoon_list/get_typhoon_list_table', data=payload )

In [75]:
#找typhoon_id
pattern = '<a class="typhoon_id" value='
deal_response = response.text
is_find = 1
ty_id = []
rp_cut = response.text.replace('\n','').replace('  ', '').split('</tr>')
rp_info = []
for i in rp_cut:
    find_del_td = 1
    while find_del_td:
        find_del_td = re.search("<td width=.*?>", i)#刪除多餘字串
        if find_del_td:
            i = i.replace(find_del_td.group(), '')
    find_del_id = re.search('<a class="typhoon_id" value=.*?>', i)
    if find_del_id:
        i = i.replace(find_del_id.group(), '').replace('</a>','')
    i = i.replace('</br>', '</td>')
    rp_info.append(i.replace('<tr>', '').split('</td>'))
ty_info_re = []
for j in rp_info: #移除一些路徑較為特殊的颱風
    if len(j) == 14:
        ty_info_re.append(j[0:13])
ty_info = []
for i in ty_info_re:
    ty_info_temp = []
    for idx, j in enumerate(i):
        if idx != 5 and idx != 6:
            j = j.replace(' ', '')
            ty_info_temp.append(j)
        else:
            ty_info_temp.append(j)
    ty_info.append(ty_info_temp)

#欄位說明
#年份, 編號, 中文名稱, 英文名稱, 侵臺路徑分類, 開始時間, 結束時間, 近臺強度, 中心最低氣壓, 中心最大風速, 最大7級暴風半徑, 最大10級暴風半徑, 警報發布數


In [76]:
'''自動化生成颱風概況網站網址'''
ty_address = []
for i in ty_info:
    info_address = 'http://rdc28.cwb.gov.tw/TDB/ntdb/pageControl/typhoon?year=' + i[0] + '&num=' + i[1] + '&name=' + i[3] +'&from_warning=true'
    ty_address.append(info_address)

In [68]:
'''自動化下載衛星雲圖'''
img_dics = {}
prop_dic = {}
ty_dic_name = ''
prop_list = ['year', 'number', 'c_name', 'en_name', 'invapath', 'alert_start', 'alert_end', 'intensity',
                  'lowest_pressure', 'highest_pressure', 'large_seven_radius', 'large_ten_radius','alert_count']
for idx, i in enumerate(ty_info):
    headers = {"X-Requested-With":"XMLHttpRequest"}
    payload = {'typhoon_year':i[0], 'typhoon_name':i[3], 'typhoon_type':'Satellite'}
    img_response = requests.post('http://rdc28.cwb.gov.tw/TDB/ctrl_typhoon_list/rtn_typhoon_product', data=payload, headers=headers)
    img_str = img_response.text.replace('\n' , '')
    if len(img_str) != 0:
        img_dic = json.loads(img_str) #將回送回來的資料用json儲存
        ty_dic_name = str(i[0]) + '_' + str(i[3]) #颱風的dictionary名稱
        img_dics.update({ty_dic_name:img_dic}) #合併所有颱風資料到一個大dic        
        prop_dic = [{'property': prop, 'value': value} for prop, value in zip(prop_list, i)]
        img_dics[ty_dic_name].update(prop_dic)

#     r = requests.get('http://rdc28.cwb.gov.tw/TDB/data/Indiv/2016/MERANTI/OBS/Satellite/Satellite_IR1_FDK.201609092350.jpg?1511463696739')
#     with open('test.jpg', 'wb') as f:
#         f.write(r.content)

In [72]:
for name in img_dics.keys(): 
    print(img_dics[name]['property'])
    if not os.path.isdir(name):#建立某颱風的資料夾
            os.mkdir(name)
    for img_style in img_dics[name]['Satellite']['subdatatype']:
        img_style_name = img_style['name']
        img_path = name + '/' + img_style_name
        if not os.path.isdir(img_path): #建立某類型的雲圖資料夾
            os.mkdir(img_path)
        img_sets = img_style['sets']
        print(img_sets)
#         img_address = 
#         with open('test.jpg', 'wb') as f:
#             f.write(r.content)

    break
    
#['GROUP', 'c_name', 'title', 'caption_has_datetime', 'subdatatype', 'e_name']

value
[['2017-09-12 00:00', 'Satellite_s1q.201709120000.jpg'], ['2017-09-12 00:10', 'Satellite_s1q.201709120010.jpg'], ['2017-09-12 00:20', 'Satellite_s1q.201709120020.jpg'], ['2017-09-12 00:30', 'Satellite_s1q.201709120030.jpg'], ['2017-09-12 00:40', 'Satellite_s1q.201709120040.jpg'], ['2017-09-12 00:50', 'Satellite_s1q.201709120050.jpg'], ['2017-09-12 01:00', 'Satellite_s1q.201709120100.jpg'], ['2017-09-12 01:10', 'Satellite_s1q.201709120110.jpg'], ['2017-09-12 01:20', 'Satellite_s1q.201709120120.jpg'], ['2017-09-12 01:30', 'Satellite_s1q.201709120130.jpg'], ['2017-09-12 01:40', 'Satellite_s1q.201709120140.jpg'], ['2017-09-12 01:50', 'Satellite_s1q.201709120150.jpg'], ['2017-09-12 02:00', 'Satellite_s1q.201709120200.jpg'], ['2017-09-12 02:10', 'Satellite_s1q.201709120210.jpg'], ['2017-09-12 02:20', 'Satellite_s1q.201709120220.jpg'], ['2017-09-12 02:30', 'Satellite_s1q.201709120230.jpg'], ['2017-09-12 02:40', 'Satellite_s1q.201709120240.jpg'], ['2017-09-12 02:50', 'Satellite_s1q.20170

In [4]:
class EPcompCrawler():
    
    BASE_URL = 'http://www.piano-e-competition.com/'

    def __init__(self, sleep_time=0.1, log=True):
        self.sleep_time = sleep_time
        self.log = log
        
    def _log_print(self, log, quite=False):
        if not quite:
            print(log)

        if self.log:
            with open("log.txt", "a") as f:
                print(log, file=f)
def _get_header():
# POST /TDB/ctrl_typhoon_list/get_typhoon_list_table/ HTTP/1.1
    header = {
'Host': 'rdc28.cwb.gov.tw',
'Connection': 'keep-alive',
'Content-Length': '22',
'Accept': 'text/plain, */*; q=0.01',
# 'Origin': 'http://rdc28.cwb.gov.tw',
'X-Requested-With': 'XMLHttpRequest',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36',
'Content-Type': 'application/x-www-form-urlencoded',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7'}
# 'Cookie': 'PHPSESSID=vvecuo9aumnaa6hrf2fquu9lr6; ci_session=a%3A5%3A%7Bs%3A10%3A%22session_id%22%3Bs%3A32%3A%2277ea4e059cb949d97e23b99c69e8ffe7%22%3Bs%3A10%3A%22ip_address%22%3Bs%3A14%3A%22210.69.218.230%22%3Bs%3A10%3A%22user_agent%22%3Bs%3A114%3A%22Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F62.0.3202.94+Safari%2F537.36%22%3Bs%3A13%3A%22last_activity%22%3Bi%3A1511449596%3Bs%3A9%3A%22user_data%22%3Bs%3A0%3A%22%22%3B%7Dc781ff204174d900932cc60821283866; TS01b0fe7f=0107dddfefb4158b847cf1d05483c6415c37bb75c11ff13b2154e507454d41d8f3d81a64830a94d79d1004eb315364ea5f61f284a1428ef062bc39e9607e4f675696a79089'}

    return header


In [212]:

r = requests.get('http://rdc28.cwb.gov.tw/TDB/data/Indiv/2016/MERANTI/OBS/Satellite/Satellite_IR1_FDK.201609092350.jpg?1511463696739')
with open('test.jpg', 'wb') as f:
    f.write(r.content)

# with open('img_test', 'w') as outfile:
#     json.dump(img_dic, outfile)


{'name': '東亞紅外線彩色雲圖',
 'sets': [['2017-08-20 00:00', 'Satellite_s1p.201708200000.jpg'],
  ['2017-08-20 00:10', 'Satellite_s1p.201708200010.jpg'],
  ['2017-08-20 00:20', 'Satellite_s1p.201708200020.jpg'],
  ['2017-08-20 00:30', 'Satellite_s1p.201708200030.jpg'],
  ['2017-08-20 00:40', 'Satellite_s1p.201708200040.jpg'],
  ['2017-08-20 00:50', 'Satellite_s1p.201708200050.jpg'],
  ['2017-08-20 01:00', 'Satellite_s1p.201708200100.jpg'],
  ['2017-08-20 01:10', 'Satellite_s1p.201708200110.jpg'],
  ['2017-08-20 01:20', 'Satellite_s1p.201708200120.jpg'],
  ['2017-08-20 01:30', 'Satellite_s1p.201708200130.jpg'],
  ['2017-08-20 01:40', 'Satellite_s1p.201708200140.jpg'],
  ['2017-08-20 01:50', 'Satellite_s1p.201708200150.jpg'],
  ['2017-08-20 02:00', 'Satellite_s1p.201708200200.jpg'],
  ['2017-08-20 02:10', 'Satellite_s1p.201708200210.jpg'],
  ['2017-08-20 02:20', 'Satellite_s1p.201708200220.jpg'],
  ['2017-08-20 02:30', 'Satellite_s1p.201708200230.jpg'],
  ['2017-08-20 02:40', 'Satellite_s1p.2017

In [213]:
requests('http://rdc28.cwb.gov.tw/TDB/data/Indiv/2016/MERANTI/OBS/Satellite/Satellite_IR1_FDK.201609092350.jpg?1511463696739')

TypeError: 'module' object is not callable

In [216]:
r = requests.get('http://rdc28.cwb.gov.tw/TDB/data/Indiv/2016/MERANTI/OBS/Satellite/Satellite_IR1_FDK.201609092350.jpg?1511463696739')

with open('test.jpg', 'wb') as f:
    f.write(r.content)

In [231]:
headers = {"X-Requested-With":"XMLHttpRequest"}
payload = {'typhoon_year':'2013', 'typhoon_name':'TRAMI', 'typhoon_type':'Satellite'}
img_response = requests.post('http://rdc28.cwb.gov.tw/TDB/ctrl_typhoon_list/rtn_typhoon_product', data=payload, headers=headers)
img_str = img_response.text.replace('\n' , '')
print(len(img_str))

70466


In [60]:
test1_dics = {'a':1,'b':2}
test2_dics = {'oko':test1_dics}
test2_dics['oko'].update({'xxx':22})
test2_dics['oko']

{'a': 1, 'b': 2, 'xxx': 22}

In [44]:
img_dics.keys()

dict_keys([])

In [79]:
prop_name_www = ['year', 'number', 'c_name', 'en_name', 'invapath', 'alert_start', 'alert_end', 'intensity',
                  'lowest_pressure', 'highest_pressure']
oxo_list = [1,2,3,4,5,6,7,8,9,10]
[{'property': prop, 'value': value} for prop, value in zip(prop_name_www, oxo_list)]


[{'property': 'year', 'value': 1},
 {'property': 'number', 'value': 2},
 {'property': 'c_name', 'value': 3},
 {'property': 'en_name', 'value': 4},
 {'property': 'invapath', 'value': 5},
 {'property': 'alert_start', 'value': 6},
 {'property': 'alert_end', 'value': 7},
 {'property': 'intensity', 'value': 8},
 {'property': 'lowest_pressure', 'value': 9},
 {'property': 'highest_pressure', 'value': 10}]